In [1]:
#baseline code for helpful prediction
from collections import defaultdict

def readJson(f):
    for l in open(f):
        yield eval(l)
### Helpfulness baseline: similar to the above. Compute the global average helpfulness rate, and the average helpfulness rate for each user

allHelpful = []
userHelpful = defaultdict(list)

for l in readJson('train.json'):
    user,item = l['reviewerID'],l['asin']
    allHelpful.append(l['helpful'])
    userHelpful[user].append(l['helpful'])

averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])
userRate = {}
for u in userHelpful:
    userRate[u] = sum([x['nHelpful'] for x in userHelpful[u]]) * 1.0 / sum([x['outOf'] for x in userHelpful[u]])

predictions = open("predictions_Helpful.txt", 'w')
for l in open("pairs_Helpful.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i,outOf = l.strip().split('-')
    outOf = int(outOf)
    if u in userRate:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*userRate[u]) + '\n')
    else:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*averageRate) + '\n')
predictions.close()


Report for Task 3:
Regression model: the helpfulness of the review can be predicted from various parameters or metadata features of users and item. But the item features wont be that useful in deciding the helpfulness of the review.
So for user features, taking deviation which is different of user rating with average item rating and length of review as more the lengthof review, more is the chance of it being helpful as well as descriptive.
Building a regression model for recommendation with userhelpfulpercent as response variable and these two parameters as independent variables.
Using the linear regression model to predict the userhelpfulpercent for pair of parameters.
    

In [2]:
# get the training dataframe with deviation of rating from items average rating and length of review from tain.json including other factors
from collections import defaultdict
import pandas as pd
import numpy as np
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sklearn.metrics as metrics
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


def readJson(f):
    for l in open(f):
        yield eval(l)
trainuser=[]
trainitem=[]
value=[]
allHelpful = []
userHelpful = defaultdict(list)
reviewtextlen=[]
allRatings=[]
itemRatings=defaultdict(list)

for l in readJson('train.json'):
    user,item = l['reviewerID'],l['asin']
    trainuser.append(l['reviewerID'])
    trainitem.append(l['asin'])
    value.append(l['overall'])
    allHelpful.append(l['helpful'])
    userHelpful[user].append(l['helpful'])
    reviewtextlen.append(len(l['reviewText']))
    allRatings.append(l['overall'])
    itemRatings[item].append(l['overall'])
    
averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])

userRate = {}
for u in userHelpful:
    userRate[u] = sum([x['nHelpful'] for x in userHelpful[u]]) * 1.0 / sum([x['outOf'] for x in userHelpful[u]])

globalAverage = sum(allRatings) / len(allRatings)

itemAverage = {}

for u in itemRatings:
    itemAverage[u] = sum(itemRatings[u]) / len(itemRatings[u])

allitems=[]
category=[]

for m in readJson('meta.json'):
    allitems.append(m['asin'])
    category.append(m['categories'])

category = map(lambda s: s.replace('[[',''), category)
category = map(lambda s: s.replace(']]',''), category)
category = map(lambda s: s.replace(",",""), category)
category = map(lambda s: s.replace("'",""), category)
category = map(lambda s: s.replace("Electronics",""), category)

# print category
# print len(category)
# print type(category[1])


itemdata={"asin":allitems,"category":category}
itemdf=pd.DataFrame(itemdata)
# print itemdf

itemAverage={"itemAverage":itemAverage}
itemAveragedf=pd.DataFrame(itemAverage)
# print itemAveragedf
itemAveragedf['asin'] = itemAveragedf.index
# print itemAveragedf
itemdfavg=pd.merge(itemdf, itemAveragedf, on='asin', how='left')
itemdfavg=itemdfavg.fillna(globalAverage)
print itemdfavg

traindata={"reviewerID":trainuser,"asin":trainitem,"overall":value,"length of rev": reviewtextlen}
traindf=pd.DataFrame(traindata)
# print traindf

userhelp={"userHelpful":userRate}
userhelpdf=pd.DataFrame(userhelp)
# print userhelpdf
userhelpdf['reviewerID'] = userhelpdf.index
# print userhelpdf

traindfhelp=pd.merge(traindf, userhelpdf, on='reviewerID', how='left')
# traindfavg['noroverall']=traindfavg['overall']-traindfavg['userAverage']
print traindfhelp


                                                     asin  \
0       ed2538e73bd60a71a3cdb5c1d7e1558fa04aa05c4856a5...   
1       805e2f92a47fa1949b03bb6eb623ba85c58fb7f252042d...   
2       24a01893dbd46c9c695b53d671fb2c461e131b1284f932...   
3       a6ed402934e3c1138111dce09256538afb04c566edf37c...   
4       8939d791e9dd035aa58da024ace69b20d651cea4adf615...   
5       ba65694f68e1385fc1f5d734747c7e95e7f122ddeff40c...   
6       6ac521636646aaecf483e148a7240f6214b7e5dc809400...   
7       d249f7b6aa564a07913dfb397aecd67b0c1b1ae8031c86...   
8       9fbcf079e4dba8e3ba33132fff2b523dc24ed38d9a4c48...   
9       fa1b01d93f61eafbdd4388b3cda5ef49572280fd163ed3...   
10      78cb1c8c60232b6d9e64bd00d8c3cd7cb3a6d8c8fdbc7b...   
11      289eef8ad63ecf5b8ed22e870bbbc0d8dbf5da64b4a9db...   
12      7a49de7024c2279e0b92ffaa6e4c783da87883dbbd8053...   
13      3473195e17b9c7023bada8063f27c618bae7842e655072...   
14      fbcf9373993e379908f1f7b68d09779766fdee6e963faf...   
15      ae42d701ca34d0d3

In [3]:

traindfavg1=pd.merge(traindfhelp, itemdfavg, on='asin', how='left')
traindfavg1['deviation']=traindfavg1['overall']-traindfavg1['itemAverage']
# print traindfavg1
#drop columns which are not required
traindfavg1.drop(['reviewerID','asin','overall','category','itemAverage'],inplace=True,axis=1)
print traindfavg1

        length of rev  userHelpful  deviation
0                 888     0.727273   0.000000
1                1929     0.954545   1.250000
2                1292     0.727273  -0.750000
3                 385     1.000000   1.250000
4                1076     1.000000  -1.750000
5                 125     1.000000   0.000000
6                 406     1.000000   0.000000
7                 181     0.500000  -0.500000
8                 275     1.000000   0.500000
9                 144     1.000000   0.500000
10                741     0.500000  -0.500000
11                309     1.000000  -1.000000
12                515     1.000000   1.000000
13                109     1.000000   0.000000
14                978     0.989362   0.000000
15                202     0.600000   0.000000
16                162     1.000000   1.166667
17                104     0.750000   1.166667
18                139     0.800000  -2.833333
19                123     1.000000   1.166667
20                219     0.875000

In [4]:
#Regression model
# create X and y

y = traindfavg1.userHelpful
X= traindfavg1.drop('userHelpful', axis=1)

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print lm.intercept_
print lm.coef_

0.714503675955
[  4.32148466e-05   4.55944828e-02]


In [7]:

# pair the feature names with the coefficients
zip(['length of rev','deviation'], lm.coef_)

[('length of rev', 4.3214846568563891e-05),
 ('deviation', 0.045594482762821284)]

In [8]:
#prepare the test data for prediction with similar two parameters ie  deviation of user rating from average item rating and length of review text
from collections import defaultdict
import pandas as pd
import numpy as np
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sklearn.metrics as metrics
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


def readJson(f):
    for l in open(f):
        yield eval(l)
helpuser=[]
helpitem=[]
helpvalue=[]
outof=[]
# allHelpful = []
# userHelpful = defaultdict(list)
reviewtextlen=[]
allRatings=[]
itemRatings=defaultdict(list)

for l in readJson('helpful.json'):
    user,item = l['reviewerID'],l['asin']
    helpuser.append(l['reviewerID'])
    helpitem.append(l['asin'])
    helpvalue.append(l['overall'])
    outof.append(l['outOf'])
#     allHelpful.append(l['helpful'])
#     userHelpful[user].append(l['helpful'])
    reviewtextlen.append(len(l['reviewText']))
    allRatings.append(l['overall'])
    itemRatings[item].append(l['overall'])
    
# averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])

# userRate = {}
# for u in userHelpful:
#     userRate[u] = sum([x['nHelpful'] for x in userHelpful[u]]) * 1.0 / sum([x['outOf'] for x in userHelpful[u]])

globalAverage = sum(allRatings) / len(allRatings)

itemAverage = {}

for u in itemRatings:
    itemAverage[u] = sum(itemRatings[u]) / len(itemRatings[u])

allitems=[]
category=[]

for m in readJson('meta.json'):
    allitems.append(m['asin'])
    category.append(m['categories'])

category = map(lambda s: s.replace('[[',''), category)
category = map(lambda s: s.replace(']]',''), category)
category = map(lambda s: s.replace(",",""), category)
category = map(lambda s: s.replace("'",""), category)
category = map(lambda s: s.replace("Electronics",""), category)

# print category
# print len(category)
# print type(category[1])


itemdata={"asin":allitems,"category":category}
itemdf=pd.DataFrame(itemdata)
# print itemdf

itemAverage={"itemAverage":itemAverage}
itemAveragedf=pd.DataFrame(itemAverage)
# print itemAveragedf
itemAveragedf['asin'] = itemAveragedf.index
# print itemAveragedf
itemdfavg=pd.merge(itemdf, itemAveragedf, on='asin', how='left')
itemdfavg=itemdfavg.fillna(globalAverage)
# print itemdfavg



helpdata={"reviewerID":helpuser,"asin":helpitem,"overall":helpvalue,"length of rev": reviewtextlen}
helpdf=pd.DataFrame(helpdata)
# print helpdf

helpdf1=pd.merge(helpdf, itemdfavg, on='asin', how='left')
helpdf1['deviation']=helpdf1['overall']-helpdf1['itemAverage']

#drop the columns not needed 
helpdf1.drop(['asin','overall','category','itemAverage'],inplace=True,axis=1)
print helpdf1


       length of rev                                         reviewerID  \
0               2175  3f8e4e55ed9642e497530b5782d4a34aed9703613d2779...   
1                113  b587571ff6586ce380f855823e4396582ed68fe82ff429...   
2                177  1e3497448fcff0a0155f43434b65d4773644b862fd0bb7...   
3                619  359fcfd1b6d053820af1cac1f2ffef7616d318d39e86c5...   
4                425  50cbcf93ce240cd355dee473773c1f37fe4cc86a837ad2...   
5               1113  7aecac1da7487eca44a3a3f0ef24d160586bacd3e839f8...   
6                184  8599932594cdda171152ded1c626809da86dbfc66ddba9...   
7                467  77c9a7fdd74e4d18be8a132e985808c7620b934c01f19b...   
8                329  5a2bf85c78cd74f1f26537e1537307ffb0b66dacdf24f2...   
9                261  25f6cdaf1a6febfb97b2210a5a2e46a11539a43ce7e075...   
10               543  daa929c58b83c82f7fe1146959d793658f6705e5b93fb2...   
11               212  1c657a832559cb064fb1c23e8177102343360a5962e31f...   
12               910  d36

In [9]:
#drop the columns not needed
helpdf1.drop(['reviewerID'],inplace=True, axis=1)
print helpdf1

       length of rev  deviation
0               2175   0.000000
1                113   0.000000
2                177   0.900000
3                619   0.900000
4                425  -0.100000
5               1113  -3.100000
6                184   0.900000
7                467  -2.100000
8                329   0.900000
9                261   0.900000
10               543   0.900000
11               212  -0.100000
12               910   0.000000
13               115   0.000000
14               104   0.000000
15               292   0.000000
16               440   0.000000
17               950  -2.000000
18               879   2.000000
19               692   1.200000
20               398  -1.800000
21               502   1.200000
22              1667   1.200000
23              1784  -1.800000
24              4267   0.625000
25               591   1.625000
26               157   1.625000
27               143  -2.375000
28               697   1.625000
29              1400  -2.375000
...     

In [11]:
# predict for a new observation
predict=lm.predict(helpdf1)


In [13]:
#check the prediction length
print len(predict)
print predict[1:10]

100000
[ 0.71938695  0.76318774  0.7822887   0.72831054  0.6212589   0.76349024
  0.6389366   0.76975639  0.76681779]


In [17]:
#append the required columns to prediction dataframe
helpdf1['predict']=predict
helpdf1['outof']=outof
helpdf1['reviewerID']=helpuser
helpdf1['asin']=helpitem
helpdf1['prediction']=helpdf1['predict']*helpdf1['outof']
print helpdf1

       length of rev  deviation  prediction  outof   predict  \
0               2175   0.000000   12.127440     15  0.808496   
1                113   0.000000    1.438774      2  0.719387   
2                177   0.900000    3.052751      4  0.763188   
3                619   0.900000    8.605176     11  0.782289   
4                425  -0.100000    0.728311      1  0.728311   
5               1113  -3.100000    4.348812      7  0.621259   
6                184   0.900000    1.526980      2  0.763490   
7                467  -2.100000    0.638937      1  0.638937   
8                329   0.900000    0.769756      1  0.769756   
9                261   0.900000    0.766818      1  0.766818   
10               543   0.900000    0.779004      1  0.779004   
11               212  -0.100000    0.719106      1  0.719106   
12               910   0.000000   10.553609     14  0.753829   
13               115   0.000000    0.719473      1  0.719473   
14               104   0.000000    0.718

In [18]:
#dictionary for fast access
predictiondict=dict(zip(helpdf1['reviewerID'],helpdf1['prediction']))

In [22]:
#writing the prediction to txt file
predictions = open("predictions_Helpful_new1.txt", 'w')
k=0
for l in open("pairs_Helpful.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i,outOf = l.strip().split('-')
    outOf = int(outOf)
    predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(helpdf1['prediction'][k]) + '\n')
    k+=1

predictions.close()
